In [47]:
import sqlite3
import pandas as pd
from datetime import date

today = date.today()
# dd/mm/YY
d1 = today.strftime("%Y%m%d")

In [34]:
d1

'20210104'

In [48]:
path = 'C:/Users/Drey/Documents/Python Scripts/TW_stock/db/stock.db'
def id_list(table):
    conn = sqlite3.connect(path)
    try:
        df = pd.read_sql(con=conn, sql=(f'SELECT DISTINCT ID FROM {table} '))
        return df['ID']
    finally:
        conn.close()

def fetch_db(stock_id, table):
    conn = sqlite3.connect(path)
    try:
        df = pd.read_sql(con=conn, sql=(f'SELECT Date,Close,Open,High,Low,Volume FROM {table} '
                                        f'WHERE ID = "{stock_id}" ORDER BY Date'))
        return df
    finally:
        conn.close()

In [7]:
industry = pd.read_csv("上市產業.csv", encoding='utf-8-sig')
ind_list = industry.Industry.unique()

In [61]:
percent_list = []
id_list = []
table = 'price'
ID = 1477   
df = fetch_db(ID, table)
close = df.Close.to_list()
percent_list.append(round(((close[-1]-close[-5])/close[-5])*100, 2))
id_list.append(ID)
df = pd.DataFrame(list(zip(id_list, percent_list)), columns =['ID', 'Percent']) 

In [62]:
df['Date'] = '20201203'

In [65]:
df['ID'] = df.ID.astype(str)

In [71]:
df

,ID,Percent,Date
0,1477,2.15,20201203


In [73]:
ind1 = pd.read_csv("產業全.csv", encoding='utf-8-sig', index_col=0)
res = pd.merge(df,ind1, on='ID')

In [91]:
res

,ID,Percent,Date,Unnamed: 0.1,Industry,Name
0,1477,2.15,20201203,100,紡織纖維,聚陽


In [89]:
def check_date(stock_id, table, d1):
    conn = sqlite3.connect(path)
    try:
        last_date = pd.read_sql(con=conn, sql=f"SELECT Date FROM {table} where ID = '{stock_id}' ORDER BY Date DESC LIMIT 1")
        print(str(last_date.iloc[0,0]))
        if str(last_date.iloc[0,0]) == d1:
            return True
        else:
            return False
    finally:
        conn.close()

In [90]:
d1 = '20201203'
stock_id = '0050'
table = 'price'
check_date(stock_id, table, d1)

20201203


True

In [ ]:
d1 = '20201203'
# table_list = ['price', 'te_price']
table_list = ['price']
percent_list = []
ID_list = []
ind1 = pd.read_csv("venv\產業全.csv", encoding='utf-8-sig', index_col=0)
# print(ind1.head())
for table in table_list:
    lstFiles = id_list(table)
    for stock_id in lstFiles[:300]:
        # print(stock_id)
        # 先確認此股票是否更新到最新日期
        if len(stock_id) == 4 and check_date(stock_id, table, d1):
            # 取得價格資訊
            df = fetch_db(stock_id, table)
            close = df.Close.to_list()
            percent_list.append(round(((close[-1]-close[-5])/close[-5])*100, 2))
            ID_list.append(stock_id)
            # if above_ma5_closeunder20(df):
            #     # 取得此股票10日內漲幅
            #     close = df.Close.to_list()
            #     percent_list.append(round(((close[-1]-close[-5])/close[-5])*100, 2))
            #     ID_list.append(stock_id)
                # s_ind = ind1[ind1.ID == stock_id].Industry.iloc[0]
                # fig_plot(df, stock_id)

high_df = pd.DataFrame(list(zip(ID_list, percent_list)), columns =['ID', 'Percent']) 
high_df['ID'] = high_df.ID.astype(str)
high_df['Date'] = d1
high_df.to_csv("venv/test.csv", encoding='utf-8-sig')
print(high_df.tail())
ind1['ID'] = ind1.ID.astype(str)
res = pd.merge(high_df,ind1, on='ID')
res.to_csv("venv/ind_high.csv", encoding='utf-8-sig')